## Before submitting
1. Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel\Restart) and then **run all cells** (in the menubar, select Cell\Run All).

2. Make sure that no assertions fail or exceptions occur, otherwise points will be subtracted.\n",

3. After you submit the notebook more tests will be run on your code. The fact that no assertions fail on your computer localy does not guarantee that completed the exercise correctly.

4. Please submit only the `*.ipynb` file.

5. Make sure you fill in any place that says `YOUR CODE HERE` or \"YOUR ANSWER HERE\". Edit only between `YOUR CODE HERE` and `END YOUR CODE`.

6. Make sure to use Python 3.6 at least.

# Sheet 5: Rounding, Overflow, Linear Algebra

In this exercise sheet, we look at various sources of numerical overflow when executing Python and numpy code for large input values, and how to efficiently handle them, for example, by using numpy special functions. There are other packages (e.g. `Decimal`) that can handle arbitrary precision but they are very slow so we tend not to use them

In [1]:
import utils
import numpy as np
import itertools
import unittest

from typing import Union
from minified import max_allowed_loops, no_imports

t = unittest.TestCase()

## Building a robust "softplus" nonlinear function (30 P)

The softplus function is defined as:

$$
\mathrm{softplus}(x) = \log(1+\exp(x)).
$$

It intervenes as elementary computation in certain machine learning models such as neural networks. Plotting it gives the following curve

![plot generated wit desmos](softplus.png)

where the function tends to **zero** for very negative input values and tends to the **identity** for very positive input values.

In [2]:
def softplus(z):
    return np.log(1 + np.exp(z))

We consider an input vector from the module `utils` containing varying values between 1 and 10000. We would like to apply the `softplus` function to all of its element in an element-wise manner.

In [3]:
X = utils.softplus_inputs
print(X)

[-10000, -1000, -800, -700, -500, -100, -10, -1, 0, 1, 10, 100, 500, 700, 800, 1000, 10000]


We choose these large values in order to test whether the behavior of the function is correct in all regimes of the function, in particular, for very small or very large values. The code below applies the `softplus` function directly to the vector of inputs and then prints for all cases the input and the corresponding function output:

In [4]:
Y = softplus(X)
for x, y in zip(X, Y):
    print(f"softplus({x:11.4f}) = {y:11.4f}")

softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -800.0000) =      0.0000
softplus(  -700.0000) =      0.0000
softplus(  -500.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(   500.0000) =    500.0000
softplus(   700.0000) =    700.0000
softplus(   800.0000) =         inf
softplus(  1000.0000) =         inf
softplus( 10000.0000) =         inf


<ipython-input-2-65fe6e33a0df>:2: RuntimeWarning: overflow encountered in exp
  return np.log(1 + np.exp(z))


For large input values, the softplus function returns `inf` whereas analysis of that function tells us that it should compute the **identity**. Let's now try to apply the softplus function one element at a time, to see whether the problem comes from numpy arrays:

In [5]:
for x in X:
    y = softplus(x)
    print(f"softplus({x:11.4f}) = {y:11.4f}")

softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -800.0000) =      0.0000
softplus(  -700.0000) =      0.0000
softplus(  -500.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(   500.0000) =    500.0000
softplus(   700.0000) =    700.0000
softplus(   800.0000) =         inf
softplus(  1000.0000) =         inf
softplus( 10000.0000) =         inf


<ipython-input-2-65fe6e33a0df>:2: RuntimeWarning: overflow encountered in exp
  return np.log(1 + np.exp(z))


Unfortunately, the result is the same. We observe that the function stops working somewhere between 700 and 800, even though the input was given in high precision `float64`.

* Create an alternative function for `softplus_robust` that applies to input scalars (int, float, etc.) and that correctly applies to values that can be much larger than 1000 (e.g. billions or more). Your function can be written in Python directly and does not need numpy parallelization.

In [6]:
@no_imports
def softplus_robust(
    x: Union[float, np.float32, np.float64]
) -> Union[float, np.float32, np.float64]:
    """
    Numerically stable implementation of softplus function. Will never 
    overflow to infinity if input is finite
    
    Args:
        x (Union[float, np.float32, np.float64]): The number of which we 
        want to calculate the softplus value
    Returns:
        Union[float, np.float32, np.float64]: softplus(x)
    """
    # YOUR CODE HERE
    if x > 0:
        sp_rob = x+np.log(np.exp(-x)+1) # not inf for x>0
    else:
        sp_rob = np.log(1+np.exp(x))    # not inf for x<=0
    return sp_rob
    # YOUR CODE HERE

In [7]:
# Verify your function
y_scalar = [softplus_robust(x) for x in X]

for x, y in zip(X, y_scalar):
    print("softplus(%11.4f) = %11.4f" % (x, y))

# the elements can be any of the three
t.assertIsInstance(y_scalar[0], (float, np.float32, np.float64))
t.assertAlmostEqual(softplus_robust(100000000), 100000000)



softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -800.0000) =      0.0000
softplus(  -700.0000) =      0.0000
softplus(  -500.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(   500.0000) =    500.0000
softplus(   700.0000) =    700.0000
softplus(   800.0000) =    800.0000
softplus(  1000.0000) =   1000.0000
softplus( 10000.0000) =  10000.0000


In [8]:
# This cell is for grading. Do not delete it

As we have seen in previous exercise sheets, the problem of functions that apply to scalars only is that they are less efficient than functions that apply to vectors directly. Therefore, we would like to handle the rounding issue directly at the vector level.

* Create a new softplus function that applies to vectors and that has the desired behavior for large input values. Your function should be fast for large input vectors. This means that you cannot use loops (including list/dict/set comprehesions) as well as the functions [`map`](https://docs.python.org/3/library/functions.html#map), [`np.vectorize`](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.vectorize.html)/[`np.fromiter`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fromiter.html)/[`np.apply_along_axis`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.apply_along_axis.html) in order to solve this exercise.

In [9]:
@max_allowed_loops(0)
@no_imports
def softplus_robust_vec(X: "vector-like"):
    """
    Vectorized version of the numericaly robust softplus function
    
    Args:
        X (vector-like): A vector (ndim=1) of values on which we want to apply the softplus function.
            It is not always a np.ndarray
        
    Returns:
        np.ndarray: A vector (ndim=1) where the ret[i] == softplus_robust(X[i])
    """
    # these are wrong!!!
    # return np.array([softplus_robust(x) for x in X])
    # return np.array(list(map(softplus_robust, X)))
    # return np.vectorize(softplus_robust)(X)
    # etc...
    # YOUR CODE HERE
    Y = np.zeros(len(X))
    X = np.array(X)
    index_pos = np.argwhere(X>0)                                # position of elements > 0 --> name index
    index_neg = np.argwhere(X<=0)                               # position of elements <= 0
    Y[index_pos] = X[index_pos]+np.log(np.exp(-X[index_pos])+1) # Y = softplus(x) for x>0
    Y[index_neg] = np.log(1+np.exp(X[index_neg]))               # Y = softplus(x) for x<=0
    return Y
    # YOUR CODE HERE

In [10]:
# Verify your function
Y = softplus_robust_vec(X)
t.assertIsInstance(Y, np.ndarray)
t.assertEqual(Y.dtype, np.float)

for tup in zip(X, Y):
    print("softplus(%11.4f) = %11.4f" % tup)

"""
This is just a demonstration.
As long as your vectorized function is consistently faster than the
loop implementation, your solution should pass. 
There are no concrete numbers about the speed-up, but as a reference
on our machines the vectorized code needs < 1ms and the one using a loop
requires > 20ms
"""
RAND_INPUT = np.random.rand(10000)
print("Vectorized function needs...")
%timeit -r2 -n5 softplus_robust_vec(RAND_INPUT)


def vectorize_with_loop(X):
    # This is a wrong implementation
    return np.array([softplus_robust(x) for x in X])

Y_loop = vectorize_with_loop(X)
np.testing.assert_allclose(Y, Y_loop)


print("Python loops need...")
%timeit -r2 -n5 vectorize_with_loop(RAND_INPUT)


softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -800.0000) =      0.0000
softplus(  -700.0000) =      0.0000
softplus(  -500.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(   500.0000) =    500.0000
softplus(   700.0000) =    700.0000
softplus(   800.0000) =    800.0000
softplus(  1000.0000) =   1000.0000
softplus( 10000.0000) =  10000.0000
Vectorized function needs...
The slowest run took 4.18 times longer than the fastest. This could mean that an intermediate result is being cached.
863 µs ± 530 µs per loop (mean ± std. dev. of 2 runs, 5 loops each)
Python loops need...
45.3 ms ± 367 µs per loop (mean ± std. dev. of 2 runs, 5 loops each)


YOUR ANSWER HERE

## Computing a partition function (40 P)



We consider a discrete probability distribution of type

$$p(\boldsymbol{x};\boldsymbol{w}) = \frac{1}{Z(\boldsymbol{w})} \exp(\boldsymbol{x}^\top \boldsymbol{w})$$

where $\boldsymbol{x} \in \{-1,1\}^{10}$ is an observation, and $\boldsymbol{w} \in \mathbb{R}^{10}$ is a vector of parameters. The term $Z(\boldsymbol{w})$ is called the partition function and is chosen such that the probability distribution sums to 1. That is, the equation:

$$\sum_{\boldsymbol{x} \in \{-1,1\}^{10}} p(\boldsymbol{x};\boldsymbol{w}) = 1$$

must be satisfied. Below is a simple method that computes the log of the partition function $Z(\boldsymbol{w})$ for various choices of parameter vectors. The considered parameters (`w_small`, `w_medium`, and `w_large`) are increasingly large (and thus problematic), and can be found in the file `utils.py`.

In [11]:
def generate_all_observations() -> np.ndarray:
    """
    All x in { -1,1 }^10 (vectors with 10 elements where each element 
    containts either -1 or 1)
    
    Returns:
        np.ndarray : All valid obvervations
    """
    return np.array(tuple(itertools.product([-1.0, 1.0], repeat=10)))


def calc_logZ(w: np.ndarray) -> float:
    """
    Calculates the log of the partition function Z
    
    Args:
        w (np.ndarray): A ten element vector (shape=(10,)) of parameters
    Returns:
        float: The log of the partition function Z
    """
    Z = np.sum(np.exp(generate_all_observations() @ w))
    return np.log(Z)


In [12]:
print(f"observations shape: {generate_all_observations().shape}")

print([round(w,2) for w in utils.w_small], f"{calc_logZ(utils.w_small):.4f}", sep=' | ')
print([round(w,2) for w in utils.w_medium], f"{calc_logZ(utils.w_medium):.4f}", sep=' | ')
print([round(w,2) for w in utils.w_big], f"{calc_logZ(utils.w_big):.4f}", sep=' | ')



observations shape: (1024, 10)
[7.3, 0.01, -0.0, -0.0, -0.54, 0.08, 4.23, 0.03, -0.0, -0.22] | 17.2457
[4.34, 1.52, 0.39, 0.14, 1.59, 18.05, -22.02, -0.0, 39.07, 0.35] | 89.5932
[-1.53, -0.0, -0.01, 5797.19, -6.65, 18924.71, -69.31, 1.12, 1.04, 116.8] | inf


<ipython-input-11-0cc07e7ea13c>:21: RuntimeWarning: overflow encountered in exp
  Z = np.sum(np.exp(generate_all_observations() @ w))


We can observe from these results, that for parameter vectors with large values (e.g. `utils.w_big`), the exponential function overflows, and thus, we do not obtain a correct value for the logarithm of `Z`.

* Implement an improved function  `getlogZ_robust` that avoids the overflow problem, and evaluates the partition function for the same parameters.

In [13]:
@no_imports
@max_allowed_loops(0)
def calc_logZ_robust(w):
    # YOUR CODE HERE
    """
    Calculates the log of the partition function Z
    
    Args:
        w (np.ndarray): A ten element vector (shape=(10,)) of parameters
    Returns:
        float: The log of the partition function Z
    """
    W = generate_all_observations() @ w
    W_max = np.max(W)
    W_red = W-W_max
    Z = np.sum(np.exp(W_red))
    return W_max + np.log(Z)
    # YOUR CODE HERE

In [14]:
# Verify your function
logZ_small = calc_logZ_robust(utils.w_small)
logZ_medium = calc_logZ_robust(utils.w_medium)
logZ_big = calc_logZ_robust(utils.w_big)
print(f"{logZ_small:.4f}")
print(f"{logZ_medium:.4f}")
print(f"{logZ_big:.4f}")

t.assertAlmostEqual(logZ_small, calc_logZ(utils.w_small))
t.assertAlmostEqual(logZ_medium, calc_logZ(utils.w_medium))

R = calc_logZ_robust(utils.w_big)
t.assertTrue(np.isfinite(R))
t.assertTrue(24919 < R < 24920)

17.2457
89.5932
24919.9913


In [15]:
# This cell is for grading. Do not delete it

* Evaluate the log-probability of the binary vectors generated by `generate_all_observations`, and return a `np.ndarray` of the indices (starting from 0) of those that have **probability** greater or equal to 0.001.

In [16]:
@no_imports
@max_allowed_loops(0)
def important_indexes(w: np.ndarray, tol: float=0.001) -> np.ndarray:
    """
    Calculates the indexes of important binary vectors for the 
    parameter vector `w`.
    
    Args:
        w (np.ndarray): The parameter vector of the partition function
        tol (float): The probability threshhold
        
    Returns:
        (np.ndarray): The indexes where the probability is greter or equal
        to `tol`
    """
    logZ = calc_logZ_robust(w)
    # YOUR CODE HERE
    W = generate_all_observations() @ w
    P = np.exp(W-logZ)
    return np.flatnonzero(P >= tol).astype(int)
    # YOUR CODE HERE

In [17]:
# Verify your function
imp_idxs = important_indexes(utils.w_big)
print(f"important indexes -> {imp_idxs}")
t.assertEqual(len(imp_idxs), 24)
t.assertEqual(imp_idxs.dtype, int)
t.assertEqual(imp_idxs[0], 81)
t.assertEqual(imp_idxs[-1], 983)

important indexes -> [ 81  83  85  87 209 211 213 215 337 339 341 343 465 467 469 471 597 599
 725 727 853 855 981 983]


## Probability of generating data from a Gaussian model (30 P)

Consider a multivariate Gaussian distribution of mean vector `m` and covariance `S`. The probability associated to a vector `x` is given by:

$$
p(\boldsymbol{x};(\boldsymbol{m},S)) = \frac{1}{\sqrt{(2\pi)^d \mathrm{det}(S)}} \exp \Big( - \frac12 (\boldsymbol{x}-\boldsymbol{m})^\top S^{-1} (\boldsymbol{x}-\boldsymbol{m})\Big)
$$

We consider the calculation of the probability of observing a certain dataset 

$$
\mathcal{D} = (\boldsymbol{x}^{(1)},\dots,\boldsymbol{x}^{(N)})
$$

assuming the data is generated according to a Gaussian distribution of fixed parameters $\boldsymbol{m}$ and $S$. Such probability density is given by the formula:

$$
\log P(\mathcal{D};(\boldsymbol{m},S)) = \log \prod_{i=1}^N p(\boldsymbol{x}^{(i)};(\boldsymbol{m},S))
$$

The function below implements such function:

In [18]:
def logp(X, m, S):
    # Find the number of dimensions from the data vector
    d = X.shape[1]

    # Invert the covariance matrix
    Sinv = np.linalg.inv(S)

    # Compute the quadratic terms for all data points
    Q = -0.5 * (np.dot(X - m, Sinv) * (X - m)).sum(axis=1)

    # Raise them quadratic terms to the exponential
    Q = np.exp(Q)

    # Divide by the terms in the denominator
    P = Q / np.sqrt((2 * np.pi) ** d * np.linalg.det(S))

    # Take the product of the probability of each data points
    Pprod = np.prod(P)

    # Return the log-probability
    return np.log(Pprod)

Evaluation of this function for various datasets and parameters provided in the file `utils.py` gives the following probabilities:

In [19]:
print(f"{logp(utils.X1, utils.m1, utils.S1):.4f}")
print(f"{logp(utils.X2, utils.m2, utils.S2):.4f}")
try :
    print(f"{logp(utils.X3, utils.m3, utils.S3):.4f}")
except OverflowError as error:
    print(error)

-24.0532
-inf
(34, 'Result too large')


<ipython-input-18-e0ec8ea46ac6>:21: RuntimeWarning: divide by zero encountered in log
  return np.log(Pprod)


This function is numerically unstable for multiple reasons. The product of many probabilities, the inversion of a large covariance matrix, and the computation of its determinant, are all potential causes for overflow. Thus, we would like to find a numerically robust way of performing each of these.

* Implement a numerically stable version of the function `logp`
* Evaluate it on the same datasets and parameters as the function `logp`

In [20]:
@no_imports
@max_allowed_loops(0)
def logp_robust(X: np.ndarray, m: np.ndarray, S: np.ndarray):
    """
    Numerically robust implemenation of `logp` function
    
    Returns:
        (float): The logp probability density
        just like in pdf
    """
    N,d = X.shape
    D_ = (X - m[np.newaxis])[0]
    A = np.linalg.solve(S,D_.T)
    S_inv = np.linalg.inv(S) # --> you can imporve inv too
    S_det = np.linalg.det(S) # --> you can improve det too
    return -1/2*(N*d*np.log(2*np.pi)+N*np.log(S_det)+np.trace(D_@A))
    # YOUR CODE HERE

In [21]:
# Verify your function
logp1 = logp_robust(utils.X1, utils.m1, utils.S1)
logp2 = logp_robust(utils.X2, utils.m2, utils.S2)
logp3 = logp_robust(utils.X3, utils.m3, utils.S3)

print(f"{logp1:.4f}")
print(f"{logp2:.4f}")
print(f"{logp3:.4f}")


outputs = np.array((logp1, logp2, logp3))
t.assertTrue(np.isfinite(outputs).all())
t.assertTrue(np.all(outputs < 0))
print("\nall log propabilities values below zero 😀")
t.assertTrue(-25 < logp1 < -24)
t.assertTrue(-2976 < logp2 < -2975)
t.assertTrue(-266011 < logp3 < -266010)

t.assertAlmostEqual(logp(utils.X1, utils.m1, utils.S1), logp1)

-24.0532
-2975.2363
-266010.0438

all log propabilities values below zero 😀
